In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import tensorflow_datasets as tfds
import os

## filepath

In [2]:
dailog_text = "研究/対話破綻/projectnextnlp-chat-dialogue-corpus/json/dailog.txt"
dir_text = "/研究/対話破綻/projectnextnlp-chat-dialogue-corpus/json/"
dir_q_t = "/研究/対話破綻/projectnextnlp-chat-dialogue-corpus/json/question/train"
dir_q_v = "/研究/対話破綻/projectnextnlp-chat-dialogue-corpus/json/question/val"
dir_a_t = "/研究/対話破綻/projectnextnlp-chat-dialogue-corpus/json/answer/train"
dir_a_v = "/研究/対話破綻/projectnextnlp-chat-dialogue-corpus/json/answer/val"

## トレーニングデータと検証データ分割

In [3]:
questions, answers = [], []
file_path = ""
for line in open(dailog_text, 'r'):
    idx = line.find('_')
    questions.append(line[:idx])
    answers.append(line[idx+1:-1])

In [4]:
train_utterance = questions[0:len(questions)-int(len(questions)*0.50)]
val_utterance = questions[int(len(questions)*0.50):]

In [5]:
train_response = answers[0:len(answers)-int(len(answers)*0.50)]
val_response = answers[int(len(answers)*0.50):]

## トークンを分かち書き

In [8]:
from janome.tokenizer import Tokenizer

In [9]:
tokenizer = Tokenizer(wakati=True)
inp_tokens, out_tokens = [], []
for i in questions:
    inp_tokens += tokenizer.tokenize(i)
for j in answers:
    out_tokens += tokenizer.tokenize(j)

In [11]:
inp_vocab_lists = list(set(inp_tokens))
out_vocab_lists = list(set(out_tokens))
tokenizaer_utteranse = tfds.features.text.TokenTextEncoder(inp_vocab_lists, tokenizer=tokenizer)
tokenizaer_response = tfds.features.text.TokenTextEncoder(out_vocab_lists, tokenizer=tokenizer)

## データをスライス

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_utterance, train_response))
val_dataset = tf.data.Dataset.from_tensor_slices((val_utterance, val_response))

In [7]:
train_dataset

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>

## データを学習できる形に持っていく

In [15]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [12]:
def encode(lang1,lang2):
    lang1 = [tokenizaer_utteranse.vocab_size] + tokenizaer_utteranse.encode(lang1.numpy()) + [tokenizaer_utteranse.vocab_size + 1]
    lang2 = [tokenizaer_response.vocab_size] + tokenizaer_response.encode(lang1.numpy()) + [tokenizaer_response.vocab_size + 1]
    return lang1,lang2

In [13]:
MAX_LENGTH = 40
def filter_max_length(x, y, max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [14]:
def tf_encoder(ut, re):
    return tf.py_function(encode, [ut,re], [tf.int64, tf.int64])

In [23]:
train_data = train_dataset.map(tf_encoder)
train_dataset = train_data.filter(filter_max_length)
train_data = train_data.cache()
train_data = train_data.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
train_data.prefetch(tf.data.experimental.AUTOTUNE)

<DatasetV1Adapter shapes: ((?, ?), (?, ?)), types: (tf.int64, tf.int64)>